<a href="https://colab.research.google.com/github/SaraAliSA/T5Week8_transformers/blob/main/Copy_of_Text_Summarization_2_gggg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning a Model for Summarization Task

In this task, you will load, preprocess, and fine-tune a T5 model on a dataset of news articles for a summarization task. Follow the steps below carefully.

### Model and Dataset Information

For this task, you will be working with the following:

- **Model Checkpoint**: Use the pre-trained model checkpoint `yalsaffar/mt5-small-Arabic-Summarization` if you face any problem you can use `google-t5/t5-small` but the first one is the correct one for both the model and tokenizer.
- **Dataset**: You will be using the `CUTD/news_articles_df` dataset. Ensure to load and preprocess the dataset correctly for training and evaluation.

**Note:**
- Any additional steps or methods you include that improve or enhance the results will be rewarded with bonus points if they are justified.
- The steps outlined here are suggestions. You are free to implement alternative methods or approaches to achieve the task, as long as you explain the reasoning and the process at the bottom of the notebook.
- You can use either TensorFlow or PyTorch for this task. If you prefer TensorFlow, feel free to use it when working with Hugging Face Transformers.
- The number of data samples you choose to work with is flexible. However, if you select a very low number of samples and the training time is too short, this could affect the evaluation of your work.

## Step 1: Load the Dataset

Load the dataset and split it into training and test sets. Use 20% of the data for testing.

In [ ]:
import pandas as pd

df = pd.read_csv("hf://datasets/CUTD/news_articles_df/news_articles_df.csv")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
!pip install datasets

In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

## Step 2: Load the Pretrained Tokenizer

Initialize a tokenizer from the gevin model checkpoint.

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("yalsaffar/mt5-small-Arabic-Summarization")

tokenizer_config.json:   0%|          | 0.00/833 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/416 [00:00<?, ?B/s]

## Step 3: Preprocess the Dataset

Define a preprocessing function that adds a prefix ("summarize:") to each input if needed and tokenizes the text for the model. The labels will be the tokenized summaries.

In [ ]:


prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summarizer"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/6702 [00:00<?, ? examples/s]

Map:   0%|          | 0/1676 [00:00<?, ? examples/s]

## Step 4: Define the Data Collator

Use a data collator designed for sequence-to-sequence models, which dynamically pads inputs and labels.

In [ ]:


from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=None, padding=True)

## Step 5: Load the Pretrained Model

Load the model for sequence-to-sequence tasks (summarization).

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("yalsaffar/mt5-small-Arabic-Summarization")

config.json:   0%|          | 0.00/896 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

## Step 6: Define Training Arguments

Set up the training configuration with parameters like learning rate, batch size, and number of epochs.

In [ ]:
from transformers import TrainingArguments, Trainer

## Step 7: Initialize the Trainer

Use the `Seq2SeqTrainer` class to train the model.

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    logging_dir="./logs",


)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## Step 8: Fine-tune the Model

In [ ]:
#!pip install --upgrade transformers

Train the model using the specified arguments and dataset.

In [ ]:


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,2.038900,1.519820


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'num_beams': 15, 'length_penalty': 0.6, 'no_repeat_ngram_size': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'num_beams': 15, 'length_penalty': 0.6, 'no_repeat_ngram_size': 2}


TrainOutput(global_step=838, training_loss=1.9688222516408114, metrics={'train_runtime': 745.2916, 'train_samples_per_second': 8.992, 'train_steps_per_second': 1.124, 'total_flos': 2918946145075200.0, 'train_loss': 1.9688222516408114, 'epoch': 1.0})

## Step 9: Inference

Once the model is trained, perform inference on a sample text to generate a summary. Use the tokenizer to process the text, and then feed it into the model to get the generated summary.

In [ ]:
text = 'في عالم اليوم المتغير بسرعة، تُعَدّ المحاكمة من أهم أعمدة تحقيق العدالة في المجتمع. من خلالها يتم فحص الأدلة، وسماع الشهادات، وتحليل الحُجَج القانونية للوصول إلى حكم نهائي يضمن حقوق الأفراد ويحافظ على النظام العام. يتطلب الأمر جهداً من القضاة، والمحامين، والشهود، والمحققين لضمان تحقيق العدالة والمساواة أمام القانون. المحاكمات ليست مجرد وسيلة للفصل في النزاعات، بل هي جزء أساسي من بناء الثقة بين المواطنين ومؤسسات الدولة. كل محاكمة تعكس مبادئ النزاهة والعدالة، وتساعد في حماية الحقوق المدنية وضمان سيادة القانون. من خلال الإجراءات المحكمة والشفافة، تسهم المحاكمات في تعزيز الأمان المجتمعي وترسيخ مبادئ العدالة التي يعتمد عليها أي نظام قانوني ناجح ومستدام.'

tokenized_text = tokenizer(text, return_tensors="pt").to('cuda')
summary = model.generate(**tokenized_text)

print(tokenizer.decode(summary[0], skip_special_tokens=True))

أصبحت المحاكمة من أهم أعمدة تحقيق العدالة في المجتمع.


In [ ]:
text =''' في عالمنا المعاصر، تمثل المحاكمة ركيزة أساسية لتحقيق العدالة وضمان حقوق الأفراد.
 من خلال المحاكمة، يتم تقديم الأدلة والشهادات وفحصها بدقة للوصول
 إلى حكم عادل ومنصف. يعمل القضاة والمحامون على تقديم حُجَج قانونية مُحكمة،
 في حين يلعب الشهود والمحققون دورًا حاسمًا في الكشف عن الحقائق.
 المحاكمات ليست مجرد أداة قانونية للفصل في النزاعا
ت، بل هي عملية تعزز الشفافية وتكرس الثقة بين المواطنين
 ومؤسسات الدولة. تُعَدّ المحاكمة وسيلة لضمان تطبيق
 القانون بالتساوي على الجميع دون تمييز، وحماية
الحقوق والحريات الأساسية. من خلال المحاكمات العادلة
والشفافة، تُبنى مجتمعات قوية تقوم على سيادة القانون
 والنزاهة في التعامل مع القضايا القانونية والجنائية'''

tokenized_text = tokenizer(text, return_tensors="pt").to('cuda')
summary = model.generate(**tokenized_text)

print(tokenizer.decode(summary[0], skip_special_tokens=True))

تمثل المحاكمة العادلة والشفافة ركيزة أساسية لتحقيق العدالة وضمان حقوق الأفراد.


In [ ]:
text = ''' اشرف رئيس الجمهوريه الباجي قايد السبسي اليوم بقصر قرطاج موكب منح الوسام
 الوطني للاستحقاق الثقافي الفنانين والمبدعين بمناسبه انعقاد ايام قرطاج السينمائيه
 والفنانون الصنف عبد الرحمان سيساكو موريتانيا جميل راتب مصر ميشال خليفي فلسطين ادريسا
 ودراغو بوركينا فاسو محمد ملص سوريا رضا الباهي تونس الحضور عمر الخليفي تونس الصنف الثاني
 عبد العزيز بن ملوكه تونس نجيب تونس منصف شرف الدين تونس الحضور الصنف الثالث ابراهيم
 تونس الخياطي تونس الحضور تونس عليها الحضور شوقي الماجري تونس الصنف الرابع كوثر بن تونس
'''

tokenized_text = tokenizer(text, return_tensors="pt").to('cuda')
summary = model.generate(**tokenized_text)

print(tokenizer.decode(summary[0], skip_special_tokens=True))

أشرف رئيس الجمهورية الباجي قايد السبسي اليوم بقصر قرطاج موكب منح الوسام الوطني للاستحقاق الثقافي الفنانين والمبدعين.
